<a href="https://colab.research.google.com/github/NataliaGon/kpi/blob/AI-cybersecurity/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Логістична регресія в системах машинного навчання

In [20]:
import os
import numpy as np
import pandas as pd
import kagglehub
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import cross_val_score

In [3]:
path = kagglehub.dataset_download("piyushrumao/malware-executable-detection")

print("Path to dataset files:", path)

100%|██████████| 27.4k/27.4k [00:00<00:00, 30.8MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/piyushrumao/malware-executable-detection/versions/2


In [4]:
df = pd.read_csv(os.path.join(path, "uci_malware_detection.csv"))

print("Size rows/columns:", df.shape)

Size rows/columns: (373, 532)


In [5]:
print(df.isnull().sum())
print(df.Label)

Label    0
F_1      0
F_2      0
F_3      0
F_4      0
        ..
F_527    0
F_528    0
F_529    0
F_530    0
F_531    0
Length: 532, dtype: int64
0      non-malicious
1      non-malicious
2      non-malicious
3      non-malicious
4      non-malicious
           ...      
368        malicious
369        malicious
370        malicious
371        malicious
372        malicious
Name: Label, Length: 373, dtype: object


Пропусків немає.

**Перевіряємо діапазон значень ознак**

In [37]:
df.drop("Label", axis=1).describe().T[["min", "max"]]

,min,max
F_1,0.0,1.0
F_2,0.0,0.0
F_3,0.0,1.0
F_4,0.0,1.0
F_5,0.0,1.0
...,...,...
F_527,0.0,1.0
F_528,0.0,1.0
F_529,0.0,1.0
F_530,0.0,1.0


Діапазон ознак від 0 до 1, тому нам не потрібен scale

**Висновки після нормалізації данних**

Ми маємо більше колонок ніж рядків, тобто ознак більше ніж прикладів, тому нам треба прибрати деякі ознаки.
Для успішного навчання кількість прикладів має в кілька разів
перевищувати кількість вхідних ознак. За малої кількості даних доводиться
штучно спрощувати структуру регресійній моделі, залишаючи найбільш
істотні ознаки.
В ідеалі данних повинно бути в 5-10 разів більше ніж ознак.

Спочатку зробимо навчання на усіх фічах, потім на обраних та звіремо результати.

## Навчання на всіх фічах

In [27]:
X = df.drop("Label", axis=1)
y = df["Label"].map({"non-malicious":0, "malicious":1})

# Ділимо на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [43]:

logReg = LogisticRegression()
# Створюємо модель
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

# Робимо передбачення
y_pred = log_reg.predict(X_test)

# Оцінка
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        61

    accuracy                           1.00        75
   macro avg       1.00      1.00      1.00        75
weighted avg       1.00      1.00      1.00        75



In [45]:
log_reg.fit(X_train, y_train)
print("Train acc:", log_reg.score(X_train, y_train))
print("Test acc:", log_reg.score(X_test, y_test))


Train acc: 1.0
Test acc: 1.0


## Навчання на обраних фічах

**Drop low-variance features**


In [29]:
selector = VarianceThreshold(threshold=0.0)
X_train_reduced = selector.fit_transform(X_train)
X_test_reduced  = selector.transform(X_test)
print(X_train_reduced.shape , X_test_reduced.shape)


(298, 502) (75, 502)


Прибрали 30 ознак

In [31]:

l1 = LogisticRegression(penalty="l1", solver="liblinear", max_iter=2000, class_weight="balanced")
l1.fit(X_train_reduced, y_train)

coefs = l1.coef_.ravel()
nonzero_mask = coefs != 0
print("Відібрано ознак:", nonzero_mask.sum(), "з", X_train_reduced.shape[1])

mask_l1 = nonzero_mask

X_train_sel = X_train_reduced[:, mask_l1]
X_test_sel  = X_test_reduced[:,  mask_l1]

print("Shapes after L1 selection:", X_train_sel.shape, X_test_sel.shape)


Відібрано ознак: 10 з 502
Shapes after L1 selection: (298, 10) (75, 10)


Регуляризація L1 виконує вибір ознак шляхом оптимізації.

Модель намагалася збалансувати:

Точність навчальних даних

Збереження малих коефіцієнтів

Виключення слабких предикторів

10 ознак достатньо для пояснення більшої частини сигналу.

In [49]:
final_lr = LogisticRegression(max_iter=2000)
final_lr.fit(X_train_sel, y_train)

y_pred = final_lr.predict(X_test_sel)
y_prob = final_lr.predict_proba(X_test_sel)[:, 1]

print("Точність:", accuracy_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob)) # false true - true false
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Точність: 1.0
ROC-AUC: 1.0
Confusion Matrix:
 [[14  0]
 [ 0 61]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        61

    accuracy                           1.00        75
   macro avg       1.00      1.00      1.00        75
weighted avg       1.00      1.00      1.00        75



**Ознаки які залишились**

In [48]:

feat_after_vt = X_train.columns[selector.get_support()]


kept_features = np.array(feat_after_vt)[mask_l1]
print("Фічі що залишились:", kept_features.tolist())


Фічі що залишились: ['F_19', 'F_20', 'F_66', 'F_69', 'F_139', 'F_140', 'F_179', 'F_291', 'F_334', 'F_447']


**Перевірка**

In [47]:
log_reg.fit(X_train, y_train)
print("Тренировочна точність:", log_reg.score(X_train, y_train))
print("Точність тесту:", log_reg.score(X_test, y_test))

Train acc: 1.0
Test acc: 1.0


## Висновки

Дані добре структуровані.

Тестова дала Accuracy = 1.0.

Це означає, що ознаки у датасеті мають чіткий сигнал, який дозволяє безпомилково відрізняти шкідливе ПЗ від безпечного.

Відбір ознак працює

Після VarianceThreshold залишилось 502 ознаки.

L1-регуляризація звела кількість до 10 ознак, і навіть на цьому піднаборі результат лишився ідеальним.

Це доводить, що більшість ознак були шумом або дублювали інформацію.

Ймовірно, у даних є “сигнатурні” ознак.

Деякі бінарні прапорці, що вказують на наявність/відсутність певного коду, можуть 100% корелювати з тим, чи файл шкідливий.

Це пояснює “занадто хорошу” якість.


Для реальних задач кібербезпеки такі дані рідко бувають настільки “чистими”.

У реальному середовищі треба очікувати появи шуму, нових варіантів шкідливого ПЗ, які не будуть настільки легко відділятися.